<a href="https://colab.research.google.com/github/aliakbarbadri/natural-language-inference/blob/master/snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

https://www.tensorflow.org/datasets/catalog/snli

https://www.tensorflow.org/tutorials/text/nmt_with_attention


# Load data

In [3]:
ds = tfds.load('snli')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/snli/plain_text/1.0.0.incomplete72H2XY/snli-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/snli/plain_text/1.0.0.incomplete72H2XY/snli-validation.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/snli/plain_text/1.0.0.incomplete72H2XY/snli-train.tfrecord


Dataset snli downloaded and prepared to /root/tensorflow_datasets/snli/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_ds = ds['train']
valid_ds = ds['validation']
test_ds = ds['test']
#labels: {0: 'entailment', 1: 'neutral', 2: 'contradiction' , -1: '-'}

In [5]:
for example in train_ds.take(1):
  print(example)

{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Washing clothes on a camping trip.'>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'A man washes or dies clothes in a primitive setting.'>}


In [6]:
def preprocess(w):
  def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  w = unicode_to_ascii(w.lower().strip())
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
# train_data = []
# train_targets = []
# valid_data = []
# valid_targets = []
# test_data = []
# test_targets = []

# for row in train_ds:
#   if int(row["label"]) == 0:
#     train_data.append(preprocess(str(row["hypothesis"].numpy())))
#     train_targets.append(preprocess(str(row["premise"].numpy())))
  

# for row in valid_ds:
#   if int(row["label"]) == 0:
#     valid_data.append(preprocess(str(row["hypothesis"].numpy())))
#     valid_targets.append(preprocess(str(row["premise"].numpy())))

for row in test_ds.take(1):
  if int(row["label"]) == 0:
    print(preprocess(row["hypothesis"].numpy().decode("utf-8")))

    print(preprocess(row["premise"].numpy().decode("utf-8")))

# print("train length", len(train_data))
# print("valid length", len(valid_data))
# print("test length", len(test_data))

<start> a girl is entertaining on stage <end>
<start> a girl in a blue leotard hula hoops on a stage with balloon shapes in the background . <end>


In [9]:
def create_dataset(num_examples):
  pairs = [[preprocess(row["hypothesis"].numpy().decode("utf-8")), preprocess(row["premise"].numpy().decode("utf-8"))]  for row in train_ds.take(num_examples) if int(row["label"])==0]
  return zip(*pairs)

In [11]:
data, target = create_dataset(20)
print(len(data))
# print(data[-1])
# print(target[-1])

2


In [12]:
def tokenize(data):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  tokenizer.fit_on_texts(data)
  tensor = lang_tokenizer.texts_to_sequences(data)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
  return tensor, tokenizer

In [13]:
def load_dataset(num_examples=100000):
  input_data, target_data = create_dataset(num_examples)
  input_tensor, input_tokenizer = tokenize(input_data)
  target_tensor, target_tokenizer = tokenize(target_data)
  return input_tensor, target_tensor, input_tokenizer, target_tokenizer

In [15]:
num_examples = 100000
input_tensor, target_tensor, input_tokenizer, target_tokenizer = load_dataset(num_examples)
max_length_target, max_length_input = target_tensor.shape[1], input_tensor.shape[1]
max_length_target, max_length_input

(84, 38)

In [16]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

26535 26535 6634 6634


In [19]:
def convert(tokenizer, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, tokenizer.index_word[t]))

In [20]:
print ("Input; index to word mapping")
convert(input_tokenizer, input_tensor_train[0])
print ()
print ("Target; index to word mapping")
convert(target_tokenizer, target_tensor_train[0])

Input; index to word mapping
1 ----> <start>
16 ----> there
7 ----> are
55 ----> dogs
15 ----> outside
4 ----> .
2 ----> <end>

Target; index to word mapping
2 ----> <start>
1 ----> a
24 ----> black
2488 ----> poodle
93 ----> jumping
8 ----> on
6 ----> the
143 ----> head
11 ----> of
1 ----> a
23 ----> white
2488 ----> poodle
26 ----> while
44 ----> walking
5 ----> in
6 ----> the
107 ----> snow
4 ----> .
3 ----> <end>


# Model-without Attention

In [ ]:
def create_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

lstm_model = create_model_lstm(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
lstm_model.summary()
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Model-Attention